## Checkpoint/Snapshot

Checkpoint to store graph state. It manages the checkpoints by threads (in the chat). That's why when run graph, you need to provide a config that contain `thread_id`

``config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)``


In [16]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    return {"foo": "b", "bar": ["b"]}


workflow = StateGraph(State)
workflow.add_node(node_a)
workflow.add_node(node_b)
workflow.add_edge(START, "node_a")
workflow.add_edge("node_a", "node_b")
workflow.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

{'foo': 'b', 'bar': ['a', 'b']}

In [2]:
print(graph.get_state(config))

StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f7a-6815-8002-69d9a10b72c8'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T06:06:04.607694+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f75-69e8-8001-cce7a15196a1'}}, tasks=(), interrupts=())


### Get state history

In [3]:
list(graph.get_state_history(config))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f7a-6815-8002-69d9a10b72c8'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T06:06:04.607694+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f75-69e8-8001-cce7a15196a1'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f75-69e8-8001-cce7a15196a1'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-07T06:06:04.605693+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f70-6be0-8000-05753a2f578c'}}, tasks=(PregelTask(id='f4afa97e-dfcf-d260-47af-cad9e0dfe45e', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

### Replay with threadId and checkpointId


In [4]:
config = {"configurable": {"thread_id": "1", "checkpoint_id": "1f073549-9f70-6be0-8000-05753a2f578c"}}
graph.invoke(None, config=config)

{'foo': 'b', 'bar': ['a', 'b']}

In [6]:
list(graph.get_state_history({"thread_id": "1"}))

[StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651f-6663-8002-d40f0614cf3a'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T06:13:07.985366+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651a-682e-8001-d392982ce672'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651a-682e-8001-d392982ce672'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-07T06:13:07.983364+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073549-9f70-6be0-8000-05753a2f578c'}}, tasks=(PregelTask(id='12e2e2d0-8f2f-9373-0abd-1161caa9dc4c', name='node_b', path=('__pregel_pull', 'node_b'), error=None, interrupts

### Update state manually

In [9]:
config = {"configurable": {"thread_id": "1"}}
graph.update_state(config, {"foo": 2, "bar": ["b"]})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f073565-b899-62a0-8003-69bff56ce72b'}}

In [10]:
list(graph.get_state_history({"thread_id": "1"}))

[StateSnapshot(values={'foo': 2, 'bar': ['a', 'b', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073565-b899-62a0-8003-69bff56ce72b'}}, metadata={'source': 'update', 'step': 3, 'parents': {}}, created_at='2025-08-07T06:18:38.860969+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651f-6663-8002-d40f0614cf3a'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'b', 'bar': ['a', 'b']}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651f-6663-8002-d40f0614cf3a'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-07T06:13:07.985366+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f073559-651a-682e-8001-d392982ce672'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'foo': 'a', 'bar': ['a']}, next=('node_b',), config={'configurable': {'thread_id':

## Memory store

If you have a requirement to share data across chat threads. You need to use the store to save them in a node and get them later in another node

Data in store grouped by namespace. Namespace is a tuple of the string list 

## Basic usage


In [1]:
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()

In [2]:
user_id = "1"
namespace_for_memory = (user_id, "memories")

In [4]:
import uuid


memory_id = str(uuid.uuid4())
memory = {"food_preference" : "I like pizza"}
in_memory_store.put(namespace_for_memory, memory_id, memory)

In [9]:
memories = in_memory_store.search(namespace_for_memory)
memories

[Item(namespace=['1', 'memories'], key='8ef1dcdf-de95-47f5-882d-03e5cd4a5e84', value={'food_preference': 'I like pizza'}, created_at='2025-08-07T06:26:36.953355+00:00', updated_at='2025-08-07T06:26:36.953355+00:00', score=None)]

In [7]:
item = in_memory_store.get(namespace_for_memory, memory_id)
item.dict()

{'namespace': ['1', 'memories'],
 'key': '8ef1dcdf-de95-47f5-882d-03e5cd4a5e84',
 'value': {'food_preference': 'I like pizza'},
 'created_at': '2025-08-07T06:26:36.953355+00:00',
 'updated_at': '2025-08-07T06:26:36.953355+00:00'}

### Semantic search

In [10]:
from langchain.embeddings import init_embeddings

store = InMemoryStore(
    index={
        "embed": init_embeddings("openai:text-embedding-3-small"),  # Embedding provider
        "dims": 1536,                              # Embedding dimensions
        "fields": ["food_preference", "$"]              # Fields to embed
    }
)

In [11]:
# Find memories about food preferences
# (This can be done after putting memories into the store)
memories = store.search(
    namespace_for_memory,
    query="What does the user like to eat?",
    limit=3  # Return top 3 matches
)

In [12]:
memories

[]

In [13]:
# Store with specific fields to embed
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {
        "food_preference": "I love Italian cuisine",
        "context": "Discussing dinner plans"
    },
    index=["food_preference"]  # Only embed "food_preferences" field
)

# Store without embedding (still retrievable, but not searchable)
store.put(
    namespace_for_memory,
    str(uuid.uuid4()),
    {"system_info": "Last updated: 2024-01-01"},
    index=False
)

In [15]:
# Find memories about food preferences
# (This can be done after putting memories into the store)
memories = store.search(
    namespace_for_memory,
    query="I love Italian cuisine",
    limit=3  # Return top 3 matches
)
memories   

[Item(namespace=['1', 'memories'], key='7f59ad29-ca2c-4860-b0d6-a5aaf97b1f0f', value={'food_preference': 'I love Italian cuisine', 'context': 'Discussing dinner plans'}, created_at='2025-08-07T06:31:22.934199+00:00', updated_at='2025-08-07T06:31:22.934199+00:00', score=0.9999992864789465),
 Item(namespace=['1', 'memories'], key='ef1793ac-d361-47bd-abe2-00a6661a4efc', value={'system_info': 'Last updated: 2024-01-01'}, created_at='2025-08-07T06:31:22.934199+00:00', updated_at='2025-08-07T06:31:22.934199+00:00', score=None)]

### Use in LangGraph

In [15]:
from langgraph.store.base import BaseStore
from langgraph.config import RunnableConfig
from langgraph.graph import MessagesState
import uuid

def update_memory(state: MessagesState, config: RunnableConfig, *, store: BaseStore):

    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "user_profile")

    # ... Analyze conversation and create a new memory

    # Create a new memory ID
    memory_id = str(uuid.uuid4())

    # We create a new memory
    store.put(namespace, memory_id, {"username": "dtt", "full_name": "Dinh Trung Thao"})

In [16]:
from langgraph.store.base import BaseStore
from langgraph.config import RunnableConfig
from langgraph.graph import MessagesState

def get_memory(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    # Get the user id from the config
    user_id = config["configurable"]["user_id"]

    # Namespace the memory
    namespace = (user_id, "user_profile")
    
    # We create a new memory
    userprofile = store.search(namespace, query={"username": "dtt", "full_name": "Dinh Trung Thao"})
    print(userprofile)

In [17]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from typing import Annotated
from typing_extensions import TypedDict
from operator import add

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]


workflow = StateGraph(State)
workflow.add_node(update_memory)
workflow.add_node(get_memory)
workflow.add_edge(START, "update_memory")
workflow.add_edge("update_memory", "get_memory")
workflow.add_edge("get_memory", END)

checkpointer = InMemorySaver()
in_memory_store = InMemoryStore()
graph = workflow.compile(checkpointer=checkpointer, store=in_memory_store)



In [18]:
# Invoke the graph
user_id = "1"
config = {"configurable": {"thread_id": "1", "user_id": user_id}}

# First let's just say hi to the AI
for update in graph.stream(
    {"messages": [{"role": "user", "content": "Hello store"}]}, config, stream_mode="updates"
):
    print(update)

{'update_memory': None}
[Item(namespace=['1', 'user_profile'], key='40e54d49-0a01-48ab-b56c-eab5b4e7cf9c', value={'username': 'dtt', 'full_name': 'Dinh Trung Thao'}, created_at='2025-08-07T07:00:29.576125+00:00', updated_at='2025-08-07T07:00:29.576125+00:00', score=None)]
{'get_memory': None}


In [27]:
config = {"configurable": {"thread_id": "1"}}
graph.invoke({"foo": ""}, config)

AttributeError: '_GeneratorContextManager' object has no attribute 'get_next_version'